### En este notebook finalmente probaré si un Topic extraction no supervisado funciona mejor que los anteriores modelos.

In [1]:
import spacy
import pandas as pd
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

import gensim.corpora as corpora
from gensim.models import LdaModel

In [2]:
nlp = spacy.load('es_core_news_md', disable = ['parser', 'ner'])
stop_words = nlp.Defaults.stop_words

def lemmatize_doc(text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV', 'PROPN']):
    text_out = [t.lemma_.lower() for t in nlp(text)
                if t.pos_ in allowed_postags
                and len(t.lemma_)>3
                and not t.is_stop]
    return text_out

def build_texts(fname):
    with open(fname) as f:
        for line in f:
            yield lemmatize_doc(line)

In [3]:
lee_data_file = 'sem_eval_train_es_topic.csv'

In [4]:
with open(lee_data_file) as f:
    for line in f:
        print(line)
        break

2018-Es-01643,"@aliciaenp Ajajjaa somos del clan twitteras perdidas pa eventos ""importantes"" "



In [5]:
texto = build_texts(lee_data_file)

In [6]:
lista_procesado = [c for c in texto]

In [7]:
len(lista_procesado)

3561

Crear diccionario

In [8]:
diccionario = corpora.Dictionary(build_texts(lee_data_file))
corpus = [diccionario.doc2bow(text) for text in build_texts(lee_data_file)]

print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]


In [9]:
len(diccionario.items())

7509

In [10]:
warnings.filterwarnings('ignore')

ldamodel = LdaModel(corpus = corpus, num_topics=11, id2word=diccionario, iterations=5000)
pprint(ldamodel.print_topics())

[(0,
  '0.017*"persona" + 0.011*"amar" + 0.009*"sentir" + 0.008*"querer" + '
  '0.007*"miedo" + 0.007*"café" + 0.007*"mujer" + 0.007*"amigo" + '
  '0.007*"conocer" + 0.006*"depresión"'),
 (1,
  '0.010*"llamar" + 0.009*"amigo" + 0.009*"amar" + 0.007*"mundo" + '
  '0.007*"muerte" + 0.007*"depresión" + 0.006*"cosa" + 0.006*"escuchar" + '
  '0.006*"palabra" + 0.006*"despertar"'),
 (2,
  '0.013*"enojo" + 0.013*"gente" + 0.012*"dejar" + 0.011*"risa" + '
  '0.009*"feliz" + 0.008*"triste" + 0.007*"volver" + 0.006*"hablar" + '
  '0.006*"tomar" + 0.005*"dolor"'),
 (3,
  '0.013*"lamentable" + 0.010*"insulto" + 0.008*"noche" + 0.008*"jajaja" + '
  '0.007*"familia" + 0.007*"infeliz" + 0.007*"dejar" + 0.006*"enfadado" + '
  '0.006*"pesadilla" + 0.006*"terrible"'),
 (4,
  '0.014*"gustar" + 0.013*"querer" + 0.011*"poner" + 0.011*"esperar" + '
  '0.010*"dormir" + 0.010*"cansado" + 0.010*"amigo" + 0.008*"lindo" + '
  '0.008*"andar" + 0.007*"cabeza"'),
 (5,
  '0.011*"sacar" + 0.011*"querer" + 0.008*"depe

In [14]:
ldamodel[corpus[0]]

[(0, 0.012996079),
 (1, 0.012995977),
 (2, 0.012996083),
 (3, 0.012995977),
 (4, 0.8700388),
 (5, 0.012995977),
 (6, 0.012995977),
 (7, 0.012996076),
 (8, 0.012995977),
 (9, 0.01299635),
 (10, 0.012996721)]

In [12]:
topic_dist_per_tweet = [ldamodel[corpus[i]] for i in range(len(corpus))]

Las predicciones del modelo LDA son difíciles de relacionar con las clasificaciones del dataframe. Por lo que dejo la siguiente estrategia para implementarla en caso de tener tiempo suficiente:
- Crear una crosstab para ver qué clase predicha se relaciona con cada topic.

<span style="color:red;">No veo interesante seguir con este notebook debido a:</span>

- La clasificación es multiclase y multietiqueta, por lo que no basta con decir el máximo, sino que habría que decidir un thresshold para decidir cuando un tweet entra en una categoría y cuando no.
- He revisado algunos tweets y no tienen ninguna relación las predicciones con las etiquetas reales. Hay varios tweet predichos como una clase que pertenecen a clases completamente diferentes.

Pese a esto, si que parece interesante el enfoque ya que pone en la misma clase palabras como pesadilla, lamentable infeliz... En otra clase persona, querer, amar, sentir...

Por lo que parece ser que no es una clasificación aleatoria, aunque creo que otros métodos supervisados serán más precisos.
